In [1]:
!pip install datasets evaluate
!pip install accelerate -U
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 4.7 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login

notebook_login()

### Load ELI5 dataset

In [3]:
from datasets import load_dataset

# eli5 = load_dataset("eli5", split="train_asks[:5000]") ## defunct: https://github.com/huggingface/datasets/issues/6605
eli5 = load_dataset("eli5_category", split="train[:5000]") ## alternative: https://github.com/huggingface/transformers/pull/28715

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for eli5_category contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/eli5_category
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.war

Generating train split:   0%|          | 0/91772 [00:00<?, ? examples/s]

Generating validation1 split:   0%|          | 0/5446 [00:00<?, ? examples/s]

Generating validation2 split:   0%|          | 0/2375 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5411 [00:00<?, ? examples/s]

In [4]:
eli5 = eli5.train_test_split(test_size=0.1)

In [5]:
len(eli5["train"]), len(eli5["test"])

(4500, 500)

In [6]:
eli5["train"][0]

{'q_id': '5r1lu5',
 'title': 'Why does it seem like something spinning starts to go in reverse as it slows down?',
 'selftext': "For example, if I spin a little desktop spinner, as it slows, the motion looks as though it stops, and then starts going backwards. The same with a car wheel while it's moving.",
 'category': 'Other',
 'subreddit': 'explainlikeimfive',
 'answers': {'a_id': ['dd3q2wf'],
  'text': ["If I have a camera taking pictures 30 times a second and a wheel with a stripe on it spinning 1 time a second you will clearly see it slowly rotating with the stripe pointing up every 30 frames. As the wheel speeds up it looks very strange on camera. If the wheel spins 30 times a second then the camera will display it as stationary because every time it takes a frame the wheel is in the same location. If the wheel spins 15 times a second you will see two stripes on opposite sides because every other frame as the stripe pointing up and every other frame has it pointing down. Now to a

### Preprocess

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [8]:
eli5 = eli5.flatten()
eli5["train"][0]

{'q_id': '5r1lu5',
 'title': 'Why does it seem like something spinning starts to go in reverse as it slows down?',
 'selftext': "For example, if I spin a little desktop spinner, as it slows, the motion looks as though it stops, and then starts going backwards. The same with a car wheel while it's moving.",
 'category': 'Other',
 'subreddit': 'explainlikeimfive',
 'answers.a_id': ['dd3q2wf'],
 'answers.text': ["If I have a camera taking pictures 30 times a second and a wheel with a stripe on it spinning 1 time a second you will clearly see it slowly rotating with the stripe pointing up every 30 frames. As the wheel speeds up it looks very strange on camera. If the wheel spins 30 times a second then the camera will display it as stationary because every time it takes a frame the wheel is in the same location. If the wheel spins 15 times a second you will see two stripes on opposite sides because every other frame as the stripe pointing up and every other frame has it pointing down. Now t

In [41]:
display(len(eli5["train"]["answers.text"]))
print("\nFirst text")
print(eli5["train"]["answers.text"][0])
print("\nSecond text")
print(eli5["train"]["answers.text"][1])
print("\nThird text")
print(eli5["train"]["answers.text"][2])

4500


First text
["If I have a camera taking pictures 30 times a second and a wheel with a stripe on it spinning 1 time a second you will clearly see it slowly rotating with the stripe pointing up every 30 frames. As the wheel speeds up it looks very strange on camera. If the wheel spins 30 times a second then the camera will display it as stationary because every time it takes a frame the wheel is in the same location. If the wheel spins 15 times a second you will see two stripes on opposite sides because every other frame as the stripe pointing up and every other frame has it pointing down. Now to answer your question. If the wheel is spinning at 29 times a second then every frame of the camera would have the line being just behind where it was in the last frame because it hadn't *quite* finished a rotation. The video would show it going backwards."]

Second text
["Money you have in your hand would be cash, or maybe it's in your bank to be used via debit card. This is what's known as cash

In [20]:
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["answers.text"]])

# def preprocess_function(examples):
#     return tokenizer(
#         [" ".join(x) for x in examples["answers.text"]],
#         truncation=True,
#         max_length=1024,
#         return_overflowing_tokens=True,
#         return_length=True
#     )

In [21]:
tokenized_eli5 = eli5.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=eli5["train"].column_names,
)

In [22]:
tokenized_eli5

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 4500
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 500
    })
})

In [23]:
block_size = 128


def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [24]:
lm_dataset = tokenized_eli5.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/4500 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/500 [00:00<?, ? examples/s]

In [25]:
lm_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 11792
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1288
    })
})

In [32]:
print(lm_dataset["train"][0]["input_ids"])
print(lm_dataset["train"][0]["attention_mask"])
print(lm_dataset["train"][0]["labels"])

[1532, 314, 423, 257, 4676, 2263, 5986, 1542, 1661, 257, 1218, 290, 257, 7825, 351, 257, 39858, 319, 340, 19493, 352, 640, 257, 1218, 345, 481, 4084, 766, 340, 6364, 24012, 351, 262, 39858, 10609, 510, 790, 1542, 13431, 13, 1081, 262, 7825, 12055, 510, 340, 3073, 845, 6283, 319, 4676, 13, 1002, 262, 7825, 37621, 1542, 1661, 257, 1218, 788, 262, 4676, 481, 3359, 340, 355, 31607, 780, 790, 640, 340, 2753, 257, 5739, 262, 7825, 318, 287, 262, 976, 4067, 13, 1002, 262, 7825, 37621, 1315, 1661, 257, 1218, 345, 481, 766, 734, 28806, 319, 6697, 5389, 780, 790, 584, 5739, 355, 262, 39858, 10609, 510, 290, 790, 584, 5739, 468, 340, 10609, 866, 13, 2735, 284, 3280, 534, 1808, 13, 1002, 262, 7825, 318, 19493]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [26]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

### Train

In [42]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained("distilgpt2")

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [43]:
training_args = TrainingArguments(
    output_dir="HF-CLM-distilgpt2",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
    ## default epochs 3 kayaknya
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,3.963300,3.806082
2,3.853300,3.797407
3,3.816900,3.795669


TrainOutput(global_step=4422, training_loss=3.877345533297645, metrics={'train_runtime': 980.0531, 'train_samples_per_second': 36.096, 'train_steps_per_second': 4.512, 'total_flos': 1155454231117824.0, 'train_loss': 3.877345533297645, 'epoch': 3.0})

In [44]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 44.51


In [45]:
trainer.push_to_hub()

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1707050706.e0d7f0a75acf.250.0:   0%|          | 0.00/6.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

events.out.tfevents.1707051695.e0d7f0a75acf.250.1:   0%|          | 0.00/359 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/rinogrego/HF-CLM-distilgpt2/commit/3586e5e2f41cf684094e7cf827d14aec3a6e46bc', commit_message='End of training', commit_description='', oid='3586e5e2f41cf684094e7cf827d14aec3a6e46bc', pr_url=None, pr_revision=None, pr_num=None)

In [49]:
training_args = TrainingArguments(
    output_dir="HF-CLM-distilgpt2-tok",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
    ## default epochs 3 kayaknya
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,3.808500,3.793663
2,3.740300,3.794556
3,3.724100,3.793600


TrainOutput(global_step=4422, training_loss=3.7519389359134725, metrics={'train_runtime': 1054.6449, 'train_samples_per_second': 33.543, 'train_steps_per_second': 4.193, 'total_flos': 1155454231117824.0, 'train_loss': 3.7519389359134725, 'epoch': 3.0})

In [50]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 44.42


In [51]:
trainer.push_to_hub()

events.out.tfevents.1707052150.e0d7f0a75acf.250.2:   0%|          | 0.00/7.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1707053214.e0d7f0a75acf.250.3:   0%|          | 0.00/359 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/rinogrego/HF-CLM-distilgpt2-tok/commit/7fdd16a1772938b2fe34f1e1c4f2982002aa7646', commit_message='End of training', commit_description='', oid='7fdd16a1772938b2fe34f1e1c4f2982002aa7646', pr_url=None, pr_revision=None, pr_num=None)

### Inference

In [46]:
prompt = "Somatic hypermutation allows the immune system to"

In [52]:
from transformers import pipeline

# generator = pipeline("text-generation", model="HF-CLM-distilgpt2")
generator = pipeline("text-generation", model="HF-CLM-distilgpt2-tok")
generator(prompt)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Somatic hypermutation allows the immune system to target certain kinds of infections, while not in overproducing. It also allows immune cells to process and respond to certain diseases. The immune system is basically a “super-fast,**-to'}]

In [53]:
## pipeline in detail

from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
tokenizer = AutoTokenizer.from_pretrained("HF-CLM-distilgpt2-tok")
inputs = tokenizer(prompt, return_tensors="pt").input_ids

from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("HF-CLM-distilgpt2-tok")
outputs = model.generate(inputs, max_new_tokens=100, do_sample=True, top_k=50, top_p=0.95)

tokenizer.batch_decode(outputs, skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Somatic hypermutation allows the immune system to prevent the spread of other diseases. I was on a team with patients with schizophrenia who had severe severe episodes of schizophrenia and also had a psychotic episode during the period leading up to the diagnosis of the illness. One patient had schizophrenia at the time but they were still living and had little information on how the illness could make people go insane or kill themselves. This was my first diagnosis of schizophrenia. All patients I was diagnosed with, in the case of the condition at the time were referred for psychiatric treatment.']

### Check Possible Next Words

Ref:
- https://stackoverflow.com/questions/76397904/generate-the-probabilities-of-all-the-next-possible-word-for-a-given-text

In [62]:
import torch
from transformers import AutoModelForCausalLM , AutoTokenizer

class LMHeadModel:

    def __init__(self, model_name):
        # Initialize the model and the tokenizer.
        self.model = AutoModelForCausalLM.from_pretrained(model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

    def get_predictions(self, sentence):
        # Encode the sentence using the tokenizer and return the model predictions.
        inputs = self.tokenizer.encode(sentence, return_tensors="pt")
        with torch.no_grad():
            outputs = self.model(inputs)
            predictions = outputs[0]
        return predictions

    def get_next_word_probabilities(self, sentence, top_k=500):

        # Get the model predictions for the sentence.
        predictions = self.get_predictions(sentence)

        # Get the next token candidates.
        next_token_candidates_tensor = predictions[0, -1, :]

        # Get the top k next token candidates.
        topk_candidates_indexes = torch.topk(
            next_token_candidates_tensor, top_k).indices.tolist()

        # Get the token probabilities for all candidates.
        all_candidates_probabilities = torch.nn.functional.softmax(
            next_token_candidates_tensor, dim=-1)

        # Filter the token probabilities for the top k candidates.
        topk_candidates_probabilities = \
            all_candidates_probabilities[topk_candidates_indexes].tolist()

        # Decode the top k candidates back to words.
        topk_candidates_tokens = \
            [self.tokenizer.decode([idx]).strip() for idx in topk_candidates_indexes]

        # Return the top k candidates and their probabilities.
        return list(zip(topk_candidates_tokens, topk_candidates_probabilities))


sentence = "I enjoy walking in the woods"
model = LMHeadModel("HF-CLM-distilgpt2-tok")
next_word_probs = model.get_next_word_probabilities(sentence, top_k=30)
len(next_word_probs)

30

In [63]:
next_word_probs

[('.', 0.22118203341960907),
 (',', 0.20596230030059814),
 ('and', 0.12315425276756287),
 ('with', 0.052421629428863525),
 ('in', 0.01910150796175003),
 ('or', 0.017990538850426674),
 ('while', 0.014477217569947243),
 ('when', 0.012884950265288353),
 ('because', 0.011704702861607075),
 ('but', 0.010909316129982471),
 ('all', 0.010906861163675785),
 ('as', 0.010380992665886879),
 ('without', 0.009774199686944485),
 ('at', 0.009436329826712608),
 ('!', 0.008912759833037853),
 ('on', 0.008273260667920113),
 ('for', 0.006886091083288193),
 ('to', 0.006041820161044598),
 ('that', 0.0051615578122437),
 ('...', 0.00455083791166544),
 ('where', 0.004411378875374794),
 ('."', 0.004253854509443045),
 ('every', 0.0042363982647657394),
 ('?', 0.003995166625827551),
 ('like', 0.0038866177201271057),
 ('looking', 0.003886247519403696),
 (';', 0.003667108016088605),
 ('around', 0.0036399171222001314),
 ('outside', 0.0036158370785415173),
 ('of', 0.003569787135347724)]